In [1]:
import cv2
import numpy as np


In [5]:
# Initialisation des détecteurs (utilise les modèles intégrés à OpenCV)
face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')
fullbody_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_fullbody.xml')

def detect_violence(video_path):
    cap = cv2.VideoCapture(video_path)
    ret, prev_frame = cap.read()
    prev_gray = cv2.cvtColor(prev_frame, cv2.COLOR_BGR2GRAY)
    violence_counter = 0
    
    while True:
        ret, frame = cap.read()
        if not ret:
            break
            
        gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
        
        # 1. Détection des visages et corps
        faces = face_cascade.detectMultiScale(gray, 1.1, 5)
        bodies = fullbody_cascade.detectMultiScale(gray, 1.1, 5)
        
        # 2. Analyse de mouvement
        flow = cv2.calcOpticalFlowFarneback(prev_gray, gray, None, 0.5, 3, 15, 3, 5, 1.2, 0)
        motion = np.mean(np.abs(flow))
        
        # 3. Règles de détection simple
        if len(faces) >= 2 and motion > 10:  # Deux visages + mouvement important
            violence_counter += 1
            cv2.putText(frame, "ALERTE: Comportement suspect!", (20,50), 
                        cv2.FONT_HERSHEY_SIMPLEX, 1, (0,0,255), 2)
        
        cv2.imshow('Detection', frame)
        prev_gray = gray.copy()
        
        if cv2.waitKey(30) == 27:  # ESC pour quitter
            break
    
    cap.release()
    cv2.destroyAllWindows()
    
    if violence_counter > 5:  # Seuil d'alertes
        return "VIDEO VIOLENTE (comportements suspects detectes)"
    else:
        return "Video normale"

# Utilisation
result = detect_violence("C:/Users/abdou/Downloads/archive (5)/Real Life Violence Dataset/Violence/V_963.mp4")  # 0 pour webcam, ou "chemin/vers/video.mp4"
print(result)


Video normale
